<a href="https://colab.research.google.com/github/MachineLearningVersusDeepLearning/Deep_Learning_Project/blob/master/03.Deep_Learning_All_In_One_Fast_Campus/05_deep_learning_latest_trend/GAN_basis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DCGAN 구현

In [1]:
 from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [2]:
!git clone https://github.com/Machi`neLearningVersusDeepLearning/ZeroToAll.git '/gdrive/My Drive/Colab Notebooks/05_deep_learning_state_of_the_art_trend/GAN'

/bin/bash: -c: line 0: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 1: syntax error: unexpected end of file


# Library 선언

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
 
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, LeakyReLU, Layer, Conv2D, BatchNormalization, Flatten, Conv2DTranspose
 
 
######## 해결 - 일반적으로 model을 쌓지 않고 class로 할때는 call 함수를 사용하여 구현함 ##############
class Generator(Model):
    def __init__(self):
        super().__init__()
        ########### 왜 7*7*512 인 것인가 #################
        self.dense = Dense(7*7*512, use_bias=False, input_shape=(100,))
        
        ########### batch normalization 이란 무엇인가 ########### 
        self.bn = BatchNormalization()
        self.lrelu = LeakyReLU()
        
        ########## conv2dtranspose 란 무엇인가 ###################
        self.deconv1 = Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False)
        self.bn1 = BatchNormalization()
        self.lrelu1 = LeakyReLU()
        
        self.deconv2 = Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False)
        self.bn2 = BatchNormalization()
        self.lrelu2 = LeakyReLU()
        
        self.deconv3 = Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same')
 
    def call(self, x, training=None, mask=None):
        h = self.lrelu(self.bn(self.dense(x), training))
        ########### 왜 reshape을 더 먼저하지 않았는가 ? ############
        h = tf.reshape(h, (-1, 7, 7, 512))
        h = self.lrelu1(self.bn1(self.deconv1(h), training))
        h = self.lrelu2(self.bn2(self.deconv2(h), training))
        return self.deconv3(h)
 
class Discriminator(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(64, (5, 5), strides=(2, 2), padding='same')
        self.bn1 = BatchNormalization()
        self.lrelu1 = LeakyReLU()
        
        self.conv2 = Conv2D(128, (5, 5), strides=(2, 2), padding='same')
        self.bn2 = BatchNormalization()
        self.lrelu2 = LeakyReLU()
        
        self.flatten = Flatten()
        self.dense = Dense(1, activation='sigmoid')
 
    def call(self, x, training=None, mask=None):
        h = self.lrelu1(self.bn1(self.conv1(x), training))
        h = self.lrelu2(self.bn2(self.conv2(h), training))
        return self.dense(h)
 
 
 
 
 
######## 데이터 불러오기 ########
 
dataset = tfds.load(name='fashion_mnist', split=tfds.Split.TRAIN)
dataset = dataset.map(lambda x: tf.cast(x['image'], tf.float32) / 255.0).batch(32)
 
####### 모델 최적화 ########
generator = Generator()
discriminator = Discriminator()
 
optim_d = tf.optimizers.Adam(1e-4)
optim_g = tf.optimizers.Adam(1e-4)
 
d_mean = tf.metrics.Mean()
g_mean = tf.metrics.Mean()
 
########### 로스 선언 #########
def discriminator_loss(d_real, d_fake):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(d_real), d_real)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(d_fake), d_fake)
    return real_loss + fake_loss
 
def generator_loss(d_fake):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(d_fake), d_fake)
 
 
 
 
########## 학습 #############
# @tf.function이란 무엇인가 ?? 
@tf.function
def train_step(image, optim_d, optim_g):
    ######## gradient tape이란 무엇인가 ? ##########
    with tf.GradientTape() as tape_d, tf.GradientTape() as tape_g: 
        z = tf.random.normal([32, 100])
        g = generator(z, training=True)
        
        d_real = discriminator(image, training=True)
        d_fake = discriminator(g, training=True)
        
        d_loss = discriminator_loss(d_real, d_fake)
        g_loss = generator_loss(d_fake)
        
        gradients_d = tape_d.gradient(d_loss, discriminator.trainable_weights)
        gradients_g = tape_g.gradient(g_loss, generator.trainable_weights)
    
    optim_d.apply_gradients(zip(gradients_d, discriminator.trainable_weights))
    optim_g.apply_gradients(zip(gradients_g, generator.trainable_weights))
    return d_loss, g_loss
 
######### 학습 시작 #############
sample_z = tf.random.normal([10, 100])
for epoch in range(100):
    for image in dataset:
        d_loss, g_loss = train_step(image, optim_d, optim_g)
 
        d_mean.update_state(d_loss)
        g_mean.update_state(g_loss)
 
    print('epoch: {}, d_loss: {}, g_loss: {}'.format(epoch+1, d_mean.result(), g_mean.result()))
    
    if epoch % 10 == 0:
        img_list = list()
        sample_img = generator(sample_z)
        for idx in range(sample_img.shape[0]):
            img_list.append(sample_img[idx][:, :, 0])
        img = np.concatenate(img_list, axis=1)
        plt.imshow(img, cmap='gray')
        plt.show()
 
    d_mean.reset_states()
    g_mean.reset_states()